<a href="https://colab.research.google.com/github/Felipe-Oliveira11/TensorFlow-CNN/blob/master/CNN_SpartialDropout2D_Conv2DTranspose.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import save_model
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
from tensorflow.keras.layers import Dense, Input, Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import SpatialDropout2D
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import Conv2D, MaxPool2D, GlobalMaxPool2D
from tensorflow.keras.callbacks import EarlyStopping

import time 
import datetime
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, accuracy_score

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
fashion_mnist = keras.datasets.fashion_mnist
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [3]:
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat','Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

In [4]:
# tipo de dados 
X_train = np.array(X_train, dtype='float64')
X_test = np.array(X_test, dtype='float64')


# padronização 
X_train = X_train / 255.0
X_test = X_test / 255.0


# image shape
image_rows = 28
image_cols = 28
image_shape = (image_rows, image_cols,1)

# Reshape | (28,28,1)
X_train = X_train.reshape(X_train.shape[0], *image_shape)
X_test = X_test.reshape(X_test.shape[0], *image_shape)

print('Image shape: {}'.format(X_train.shape))

Image shape: (60000, 28, 28, 1)


In [5]:
X_test.shape

(10000, 28, 28, 1)

<br>
<br>


<hr>
<br>


### SpatialDropout2D 

SpatialDroput2D é uma técnica derivada do Dropout padrão aplicado normalmente em Redes MultiLayers perceptron, mas o grande diferencial do Dropout padrão para o Spatial é que, o Spatial elimina um Feature Map por inteiro e está melhor alinhado para as camadas convolucionais, pois ele vai dropar alguns filtros por inteiro e não somente alguns pixels, os pixels das imagens são altamente correlacionados o que interfere se aplicarmos o Dropout normal, a vantagem do Spatial é que são retirados os filtros por inteiros e não somente pixels de forma aleatória. 


Vamos aplicar o SpatialDropout2D em nossa arquitetura de Rede neural convolucional e ver os resultados. 


<br>

In [6]:
# CNN com SpatialDropout2D

model = Sequential()
model.add(Input(shape=(image_shape)))
model.add(Conv2D(32, (3,3), activation='relu', padding='same'))
model.add(SpatialDropout2D(rate=0.30))  # Spatial 1 
model.add(Conv2D(32, (3,3), activation='relu', padding='same'))
model.add(MaxPool2D(2,2))
model.add(Conv2D(64, (3,3), activation='relu', padding='same'))
model.add(SpatialDropout2D(0.30)) # Spatial 2 
model.add(Conv2D(64, (3,3), activation='relu', padding='same'))
model.add(MaxPool2D(2,2))
model.add(Conv2D(128, (3,3), activation='relu', padding='valid'))
model.add(MaxPool2D(2,2))
model.add(GlobalMaxPool2D())
model.add(Dense(128, activation='relu'))
model.add(Dense(10, activation='softmax'))


In [7]:
# compile 
model.compile(optimizer=Adam(learning_rate=0.001),
              loss=SparseCategoricalCrossentropy(),
              metrics=['accuracy'])


# EarlyStopping 
early_stopping = EarlyStopping(monitor='val_loss',
                               mode='min',
                               min_delta=0.01,
                               patience=5
                               )

# treino 
model.fit(X_train, y_train,
          epochs=20,
          batch_size=64,
          validation_data=(X_test, y_test),
          callbacks=[early_stopping])

Epoch 1/20
938/938 [==============================] - 242s 258ms/step - loss: 0.5946 - accuracy: 0.7814 - val_loss: 0.3658 - val_accuracy: 0.8670
Epoch 2/20
938/938 [==============================] - 243s 259ms/step - loss: 0.3428 - accuracy: 0.8753 - val_loss: 0.3071 - val_accuracy: 0.8880
Epoch 3/20
938/938 [==============================] - 244s 260ms/step - loss: 0.2857 - accuracy: 0.8968 - val_loss: 0.2740 - val_accuracy: 0.9031
Epoch 4/20
938/938 [==============================] - 241s 257ms/step - loss: 0.2565 - accuracy: 0.9058 - val_loss: 0.2553 - val_accuracy: 0.9086
Epoch 5/20
938/938 [==============================] - 244s 260ms/step - loss: 0.2385 - accuracy: 0.9136 - val_loss: 0.2501 - val_accuracy: 0.9094
Epoch 6/20
938/938 [==============================] - 244s 260ms/step - loss: 0.2192 - accuracy: 0.9196 - val_loss: 0.2545 - val_accuracy: 0.9069
Epoch 7/20
938/938 [==============================] - 244s 260ms/step - loss: 0.2093 - accuracy: 0.9230 - val_loss: 0.2571 -

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 32)        320       
_________________________________________________________________
spatial_dropout2d (SpatialDr (None, 28, 28, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 32)        9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 64)        18496     
_________________________________________________________________
spatial_dropout2d_1 (Spatial (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 14, 14, 64)        3

In [9]:
model.evaluate(X_test, y_test)

313/313 [==============================] - 9s 30ms/step - loss: 0.2446 - accuracy: 0.9180


[0.24456970393657684, 0.9179999828338623]

<br>
<br>
<hr>
<br>


### Convolução Transposta

A convolução transposta é a operação de convolução que tem por objetivo realizar o UpSampling das imagens, é uma camada muito utilizada para Super-Resolução e também segmantação de imagens. 

Vou procurar aplicar como um teste, esta camada em nossa arquitetura e ver os resultados finais, aplicado a classificação de imagens. 



<br>
<hr>

In [10]:
from tensorflow.keras.layers import Conv2DTranspose

In [11]:
# (Alterar Strides da Convolução Transposta)

model = Sequential()
model.add(Input(shape=(image_shape)))
model.add(Conv2D(32, (3,3), activation='relu', padding='same'))
model.add(SpatialDropout2D(rate=0.30))
model.add(Conv2D(32, (3,3), activation='relu', padding='same'))
model.add(MaxPool2D(2,2))
model.add(Conv2D(64, (3,3), activation='relu', padding='same'))
model.add(Conv2D(64, (3,3), activation='relu', padding='same'))
model.add(MaxPool2D(2,2))
model.add(Conv2D(128, (3,3), activation='relu', padding='valid'))
model.add(MaxPool2D(2,2))
model.add(Conv2DTranspose(128, (3,3)))
model.add(Conv2DTranspose(64, (3,3)))
model.add(Conv2DTranspose(64, (3,3)))
model.add(Conv2DTranspose(32, (3,3)))
model.add(Conv2DTranspose(32, (3,3)))
model.add(GlobalMaxPool2D())
model.add(Dense(128, activation='relu'))
model.add(Dense(10, activation='softmax'))

In [12]:
# compile 
model.compile(optimizer=Adam(learning_rate=0.001),
              loss=SparseCategoricalCrossentropy(),
              metrics=['accuracy'])


# EarlyStopping 
early_stopping = EarlyStopping(monitor='val_loss',
                               mode='min',
                               min_delta=0.001,
                               patience=10
                               )

# treino 
model.fit(X_train, y_train,
          epochs=30,
          batch_size=64,
          validation_data=(X_test, y_test),
          callbacks=[early_stopping])

Epoch 1/30
938/938 [==============================] - 319s 340ms/step - loss: 0.5737 - accuracy: 0.7886 - val_loss: 0.4001 - val_accuracy: 0.8544
Epoch 2/30
938/938 [==============================] - 321s 342ms/step - loss: 0.3355 - accuracy: 0.8772 - val_loss: 0.3880 - val_accuracy: 0.8648
Epoch 3/30
938/938 [==============================] - 318s 339ms/step - loss: 0.2868 - accuracy: 0.8959 - val_loss: 0.3115 - val_accuracy: 0.8930
Epoch 4/30
938/938 [==============================] - 327s 348ms/step - loss: 0.2576 - accuracy: 0.9070 - val_loss: 0.2868 - val_accuracy: 0.8990
Epoch 5/30
938/938 [==============================] - 342s 365ms/step - loss: 0.2403 - accuracy: 0.9122 - val_loss: 0.2965 - val_accuracy: 0.9006
Epoch 6/30
938/938 [==============================] - 345s 368ms/step - loss: 0.2271 - accuracy: 0.9182 - val_loss: 0.2345 - val_accuracy: 0.9139
Epoch 7/30
938/938 [==============================] - 345s 368ms/step - loss: 0.2115 - accuracy: 0.9237 - val_loss: 0.2554 -

In [13]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 28, 28, 32)        320       
_________________________________________________________________
spatial_dropout2d_2 (Spatial (None, 28, 28, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 28, 28, 32)        9248      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 14, 14, 64)        18496     
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 14, 14, 64)        36928     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 7, 7, 64)         

In [14]:
model.evaluate(X_test, y_test)

313/313 [==============================] - 17s 55ms/step - loss: 0.2766 - accuracy: 0.9177


[0.2765956223011017, 0.9176999926567078]